In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
# import xlsx, decode cell color to new columns
path = '../Results/model_answers_manual_eval.xlsx'
df = pd.read_excel(path)

In [55]:
df

,Unnamed: 0,#,2022 GTMI Indicators & Sub-indicators NL,Response options & Data format NL,ground_truth,ooo,ooF,oCo,Poo,oCF,PCo,PoF,PCF
0,0,I-1,Is er een gedeeld cloud platform beschikbaar v...,"0= Nee, 1= Alleen cloud strategie/beleid (nog ...",2,2 (Ja),Informatie over de beschikbaarheid van een ...,Op basis van de bovenstaande context en infor...,2 (ja),Er zijn verschillende cloud service providers...,0 (Nee),1,0 (Nee). Er is geen gedeeld cloud platform ...
1,1,I-1.1,Naam van het Overheids Cloud platform,Tekst,OpenStack,"Ja, het overheids cloud platform dat beschikb...","Ja, er is een gedeeld cloud platform beschikb...",Het antwoord op de vraag of er een gedeeld cl...,[Naam van het Overheids Cloud platform] Eind...,NaN,Geen antwoord.,Geen antwoord.,Er bestaan rijksbrede contractvoorwaarden vo...
2,2,I-1.2,Cloud platform / strategie URL,URL,https://joinup.ec.europa.eu/collection/open-so...,Voor een gedeeld cloud platform dat beschikba...,"Ja, er is een gedeeld cloud platform beschikb...",De gedeelde cloud platform strategie voor ove...,[URL van het Nederlandse overheids cloud plat...,"Nee, er is geen gedeeld cloud platform besch...",Geen antwoord.,Geen antwoord.,https://www.cloudplatform.nl/nl/cloud-aanbod...
3,3,I-1.3,Overheids Cloud gelanceerd / zal worden gelanc...,YYYY,2015,De Overheids Cloud is een gedeeld cloud plat...,"Ja, de Overheids Cloud is gelanceerd in 2016.",De Rijksoverheid heeft zich gecommitteerd aan...,2023 --- Deze simulatie illustreert hoe een...,Er bestaan rijksbrede contractvoorwaarden voo...,Geen antwoord.,2023,Geen antwoord.
4,4,I-1.4,Type beschikbaar cloud platform,"0= Onbekend, 1= Publiek (Commercieel), 2= Priv...",1,"Om een nauwkeurig antwoord te geven, zou er...",Er zijn meerdere gedeelde cloud platforms bes...,Gezien de informatie uit de context en de uit...,3 [Einde van de simulatie],Onbekend Publiek (Commercieel) Privaat (Ov...,2 (Publiek (Overheid)),3,"0= Onbekend, 1= Publiek (Commercieel), 2= Pr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,347,I-48.6,Is er een aanbestedingsbeleid gericht op het p...,"0= Nee, 1= Ja",0,"0= Nee, er is geen specifiek overheidsbeleid ...","0= Nee, 1= Ja","0= Nee, 1= Ja Antwoord volgens het data-forma...",1,"Ja, er is een overheidsbeleid ter ondersteun...",1 (Let op: Dit antwoord is alleen gebaseerd o...,1,1
348,348,I-48.6.1,Indien aanbestedingsbeleid gericht op priorite...,Voer URL in (publieke link) of Voeg relevant r...,-,Om een nauwkeurig antwoord te geven op uw vra...,"Ja, er is een overheidsbeleid ter ondersteuni...",Om een nauwkeurig antwoord te geven op uw vra...,[URL/Rapport/Document] Let op: Als de AI-ass...,Er is geen specifiek overheidsbeleid ter ond...,"Geen antwoord. De vraag ""Is er een overheids...",Geen antwoord.,Er is geen overheidsbeleid ter ondersteuning...
349,349,I-48.7,Publicatie van de resultaten/voortgang in de o...,"0= Nee, 1= Ja (intern, niet gepubliceerd), 2=...",2,"Ja, er is een overheidsbeleid ter ondersteun...",Voor de ondersteuning van GovTech-startups e...,"2 (Publiek, gepubliceerd) De rijksoverheid he...",2 Dit antwoord is gebaseerd op de huidige be...,"Ja, er is een overheidsbeleid ter ondersteun...","2= Ja (publiek, gepubliceerd)","2 (publiek, gepubliceerd)","Ja (publiek, gepubliceerd) Bron: Innovati..."
350,350,I-48.7.1,Indien publicatie van de resultaten/voortgang ...,Voer URL in (publieke link) of Voeg relevant r...,https://www.government.nl/latest/news/2021/04/...,"Ja, in Nederland is er het ""Actieplan Digital...","Ja, het kabinet stimuleert de ontwikkeling va...",Om een specifiek antwoord te geven op uw vraa...,[Alleen de URL of het document als het bestaa...,In het data-format met behulp van de context...,<|assistant|> Geen antwoord.,Voer URL in (publieke link) of Voeg relevant ...,Geen antwoord.


In [56]:
import openpyxl
from openpyxl import load_workbook

In [57]:
wb = load_workbook(path, data_only=True)

In [58]:
sh = wb['Scoring_eval']

In [59]:
sh['A1'].value

In [60]:
color_in_hex = sh['F3'].fill.start_color.index # this gives you Hexadecimal value of the color

In [61]:
color_in_hex

9

In [74]:
# extract color from each cell of a column, and store in a list

def get_colour_list(column: str) -> list:
    color_list = []
    for i in range(2, 353):
        color_in_hex = sh[f'{column}{i}'].fill.start_color.index
        color_list.append(color_in_hex)
    return color_list

In [63]:
ooo_colours = get_colour_list(column='F')
ooF_colours = get_colour_list(column='G')
oCo_colours = get_colour_list(column='H')
Poo_colours = get_colour_list(column='I')
oCF_colours = get_colour_list(column='J')
PCo_colours = get_colour_list(column='K')
PoF_colours = get_colour_list(column='L')
PCF_colours = get_colour_list(column='M')

In [64]:
df_colours = pd.DataFrame(
    {'ooo': ooo_colours, 
     'ooF': ooF_colours, 
     'oCo': oCo_colours, 
     'Poo': Poo_colours, 
     'oCF': oCF_colours, 
     'PCo': PCo_colours, 
     'PoF': PoF_colours, 
     'PCF': PCF_colours}
)

In [65]:
df_colours

,ooo,ooF,oCo,Poo,oCF,PCo,PoF,PCF
0,6,5,5,6,8,9,9,5
1,9,9,9,5,00000000,00000000,00000000,9
2,9,5,9,5,5,00000000,00000000,9
3,5,9,5,9,5,00000000,9,00000000
4,5,5,5,9,5,9,9,5
...,...,...,...,...,...,...,...,...
346,5,5,6,6,5,6,5,5
347,6,5,5,9,8,9,9,9
348,5,5,5,5,5,00000000,00000000,5
349,8,8,6,6,8,6,6,6


In [66]:
# create dict to recode color codes to meaningful names
color_to_meaning = {6: 'correct', 
                    5: 'incorrect',
                    8: 'correct, not following format',
                    9: 'incorrect, but following format',
                    '00000000': 'no answer given',
                    1: 'not enough context for models to answer (external framework score)',
                    }

In [67]:
# recode color codes to meaningful names
df_colours = df_colours.replace(color_to_meaning)

In [68]:
df_colours

,ooo,ooF,oCo,Poo,oCF,PCo,PoF,PCF
0,correct,incorrect,incorrect,correct,"correct, not following format","incorrect, but following format","incorrect, but following format",incorrect
1,"incorrect, but following format","incorrect, but following format","incorrect, but following format",incorrect,no answer given,no answer given,no answer given,"incorrect, but following format"
2,"incorrect, but following format",incorrect,"incorrect, but following format",incorrect,incorrect,no answer given,no answer given,"incorrect, but following format"
3,incorrect,"incorrect, but following format",incorrect,"incorrect, but following format",incorrect,no answer given,"incorrect, but following format",no answer given
4,incorrect,incorrect,incorrect,"incorrect, but following format",incorrect,"incorrect, but following format","incorrect, but following format",incorrect
...,...,...,...,...,...,...,...,...
346,incorrect,incorrect,correct,correct,incorrect,correct,incorrect,incorrect
347,correct,incorrect,incorrect,"incorrect, but following format","correct, not following format","incorrect, but following format","incorrect, but following format","incorrect, but following format"
348,incorrect,incorrect,incorrect,incorrect,incorrect,no answer given,no answer given,incorrect
349,"correct, not following format","correct, not following format",correct,correct,"correct, not following format",correct,correct,correct


In [69]:
# make table with the value counts of all columns
categories = df_colours.apply(pd.Series.value_counts, normalize=True).fillna(0)

In [70]:
categories

,ooo,ooF,oCo,Poo,oCF,PCo,PoF,PCF
correct,0.199430,0.096866,0.150997,0.273504,0.128205,0.207977,0.279202,0.094017
"correct, not following format",0.014245,0.031339,0.028490,0.000000,0.062678,0.000000,0.005698,0.017094
incorrect,0.555556,0.618234,0.589744,0.245014,0.572650,0.136752,0.142450,0.524217
"incorrect, but following format",0.216524,0.236467,0.216524,0.396011,0.207977,0.287749,0.356125,0.153846
no answer given,0.000000,0.002849,0.000000,0.071225,0.014245,0.353276,0.202279,0.196581
not enough context for models to answer (external framework score),0.014245,0.014245,0.014245,0.014245,0.014245,0.014245,0.014245,0.014245


In [71]:
categories = categories.T

In [72]:
# entries are correct if they are either 'correct' or 'correct, not following format'. Calculate the accuracy of each column (NOT ENOUGH CONTEXT NOT TAKEN INTO ACCOUNT)
categories['accuracy'] = ( categories['correct'] + categories['correct, not following format'] ) / (categories['correct'] + categories['correct, not following format'] + categories['incorrect'] + categories['incorrect, but following format'] + categories['no answer given'] )

In [73]:
categories

,correct,"correct, not following format",incorrect,"incorrect, but following format",no answer given,not enough context for models to answer (external framework score),accuracy
ooo,0.199430,0.014245,0.555556,0.216524,0.000000,0.014245,0.216763
ooF,0.096866,0.031339,0.618234,0.236467,0.002849,0.014245,0.130058
oCo,0.150997,0.028490,0.589744,0.216524,0.000000,0.014245,0.182081
Poo,0.273504,0.000000,0.245014,0.396011,0.071225,0.014245,0.277457
oCF,0.128205,0.062678,0.572650,0.207977,0.014245,0.014245,0.193642
PCo,0.207977,0.000000,0.136752,0.287749,0.353276,0.014245,0.210983
PoF,0.279202,0.005698,0.142450,0.356125,0.202279,0.014245,0.289017
PCF,0.094017,0.017094,0.524217,0.153846,0.196581,0.014245,0.112717
